# Collaborative filtering explained using Books-Crossing dataset

In [4]:
import os
import pandas as pd
import math
import matplotlib.pyplot as plt
import scipy.stats as stats
import sqlite3
import numpy as np
import seaborn as sns; sns.set(style="white", color_codes=True)
import utilities as u
import warnings 
%matplotlib inline
warnings.filterwarnings("ignore")

In [34]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Grocery_and_Gourmet_Food_5.json.gz')


In [35]:
df.head()

,reviewerID,asin,reviewerName,helpful,unixReviewTime,reviewText,overall,reviewTime,summary
0,A1VEELTKS8NLZB,616719923X,Amazon Customer,"[0, 0]",1370044800,Just another flavor of Kit Kat but the taste i...,4.0,"06 1, 2013",Good Taste
1,A14R9XMZVJ6INB,616719923X,amf0001,"[0, 1]",1400457600,I bought this on impulse and it comes from Jap...,3.0,"05 19, 2014","3.5 stars, sadly not as wonderful as I had hoped"
2,A27IQHDZFQFNGG,616719923X,Caitlin,"[3, 4]",1381190400,Really good. Great gift for any fan of green t...,4.0,"10 8, 2013",Yum!
3,A31QY5TASILE89,616719923X,DebraDownSth,"[0, 0]",1369008000,"I had never had it before, was curious to see ...",5.0,"05 20, 2013",Unexpected flavor meld
4,A2LWK003FFMCI5,616719923X,Diana X.,"[1, 2]",1369526400,I've been looking forward to trying these afte...,4.0,"05 26, 2013","Not a very strong tea flavor, but still yummy ..."


In [43]:
len(df['asin'].unique())

8713

In [47]:
df.groupby(['reviewerID','asin']).sum()

unixReviewTime  overall
reviewerID            asin                               
A00177463W0XWB16A9O05 B0029XDZIK      1357430400      5.0
                      B003C4YIFE      1361232000      5.0
                      B003YUW7EK      1364774400      4.0
                      B00474OR8G      1364774400      4.0
                      B007OSBE1U      1357430400      5.0
                      B007OSBEVA      1357430400      3.0
                      B007OSBFY6      1357430400      4.0
                      B007TGDXMK      1364774400      5.0
                      B0084I1ZYU      1361923200      5.0
                      B008I1XLDQ      1357430400      5.0
                      B008TKL6GA      1361923200      5.0
                      B0094ISOMA      1361923200      4.0
                      B00BFM6OAW      1364774400      3.0
A022899328A0QROR32DCT B000EVE3Y4      1404864000      5.0
                      B001ACMCNU      1357430400      1.0
                      B003TO9RSU      1363392000      4.0
                      B003V8QGAG      1363392000      5.0
                      B003VIJI1A      1356739200      5.0
                      B003VIJI38      1362441600      5.0
                      B003VKR0HM      1357776000      1.0
                      B003YBH398      1386633600      2.0
                      B00CMQDKES      1386633600      3.0
                      B00DBWU2JS      1395878400      2.0
A04309042SDSL8YX2HRR7 B000B6J51I      1367107200      5.0
                      B000EM6Q34      1386201600      2.0
                      B000FYXBPW      1374105600      4.0
                      B003VN74VQ      1367193600      4.0
                      B0044QY728      1365897600      4.0
A068255029AHTHDXZURNU B000HKFF94      1405036800      5.0
                      B000K8WVYA      1405036800      5.0
...                                          ...      ...
AZXKAH2DE6C8A         B001VNGJ5K      1281744000      5.0
                      B0029JW8T6      1281744000      5.0
                      B002SIDKPE      1393891200      5.0
                      B003WFNW32      1309132800      5.0
                      B003YEWVRE      1346889600      5.0
                      B004AE1FHY      1364342400      5.0
                      B004OCCL7K      1338940800      5.0
                      B004TPKAN4      1364342400      5.0
                      B004X8TJP2      1393891200      5.0
                      B006H34CUS      1396742400      5.0
                      B007W14RMM      1393891200      5.0
                      B007W14RN6      1393891200      5.0
AZXON596A1VXC         B00113SKZW      1251936000      5.0
                      B00113ZTVK      1253836800      5.0
                      B001EO5RCM      1360281600      5.0
                      B001EO5S0I      1360281600      5.0
                      B001L4JH5I      1254528000      5.0
                      B00271QQ7Q      1360281600      5.0
                      B0028UA2NA      1360281600      5.0
AZYXC63SS008M         B000NGNEKY      1379548800      5.0
                      B0039LVLS2      1317254400      5.0
                      B0040QCJDG      1379548800      5.0
                      B0040WCQKQ      1317254400      5.0
                      B0054TWPNC      1379548800      5.0
                      B00AKVU5CW      1379548800      5.0
AZZ5ASC403N74         B0035N3ADS      1358380800      5.0
                      B004FEN3GA      1303862400      2.0
                      B004U49QU2      1308787200      3.0
                      B00BIEU5PC      1375574400      3.0
                      B00H72831M      1401321600      4.0

[151254 rows x 2 columns]